In [1]:
import sys
sys.path.append('/kaggle/input/timm-pytorch-image-models/pytorch-image-models-master/')
sys.path.append('../input/hubmap-coat') 
sys.path.append("../input/segmentation-models-pytorch/segmentation_models.pytorch-0.2.1/")
sys.path.append("../input/pretrainedmodels/pretrainedmodels-0.7.4")
sys.path.append("../input/efficientnet-pytorch/EfficientNet-PyTorch-master")
sys.path.append('/kaggle/input/hubmap-coat/')
!pip install -qq /kaggle/input/mmdetection/einops-0.4.1-py3-none-any.whl

In [2]:
import cv2
import os 
import sys
import segmentation_models_pytorch as smp
import tifffile as tiff
import pandas as pd
from torch import nn
import torch
import torch.cuda.amp as amp
import torch.nn.functional as F
from coat import *
from daformer import *
from helper import *
import torch
import torch.nn as nn

In [3]:
valid_file = '../input/hubmap-organ-segmentation/test.csv'
tiff_dir   = '../input/hubmap-organ-segmentation/test_images'

valid_df = pd.read_csv(valid_file)
valid_df.loc[:,'img_area']=valid_df['img_height']*valid_df['img_width']
valid_df = valid_df.sort_values('img_area').reset_index(drop=True)

image_size_list  = [1024]
organ_threshold = {
    'Hubmap': {
        'kidney'        : 0.40,
        'prostate'      : 0.40,
        'largeintestine': 0.40,
        'spleen'        : 0.40,
        'lung'          : 0.10,
    },
    'HPA': {
        'kidney'        : 0.50,
        'prostate'      : 0.50,
        'largeintestine': 0.50,
        'spleen'        : 0.50,
        'lung'          : 0.10,
    },
}
data_source =['Hubmap', 'HPA']
organ = ['kidney', 'prostate', 'largeintestine', 'spleen', 'lung']

In [4]:
def swa_weight(model,
              checkpoint_list,
              weight_list):
    checkpoints = [torch.load(f)['state_dict'] for f in checkpoint_list]
    for name, param in model.named_parameters():
            param.data = sum([ckpt[name] * w for ckpt, w in zip(checkpoints, weight_list)])
    return model

In [6]:
def load_net(Net,encoder,ckpt_list,weight_list):
    model = Net(encoder=encoder).cuda()
    model = swa_weight(model,ckpt_list,weight_list)
    return model

In [8]:
class Net_lite_medium(nn.Module):
      def __init__(self,encoder,decoder=daformer_conv3x3,   
                    encoder_cfg={},
                    decoder_cfg={},
                    ):  # decoder = daformer_conv3x3,   for coat-medium
        super(Net_lite_medium, self).__init__()
        decoder_dim = decoder_cfg.get('decoder_dim', 320)

        # ----
        self.output_type = ['inference', 'loss']


        self.rgb = RGB()



        self.encoder = encoder

        encoder_dim = self.encoder.embed_dims
        self.decoder = decoder(
          encoder_dim=encoder_dim,
          decoder_dim=decoder_dim,
        )
        self.logit = nn.Sequential(
          nn.Conv2d(decoder_dim, 1, kernel_size=1),
        )
        self.aux = nn.ModuleList([
                nn.Conv2d(decoder_dim, 1, kernel_size=1, padding=0) for i in range(4)
            ])


      def forward(self, batch):

        x = batch['image']
        num_class = 5
        x = self.rgb(x)
        B, C, H, W = x.shape
        encoder = self.encoder(x)
        last, decoder = self.decoder(encoder)

        logit = self.logit(last)
        logit = F.interpolate(logit, size=None, scale_factor=4, mode='bilinear', align_corners=False)

        output = {}
        if 'loss' in self.output_type:
          output['bce_loss'] = F.binary_cross_entropy_with_logits(logit,batch['mask'])
          for i in range(4):
            output['aux%d_loss'%i] = criterion_aux_loss(self.aux[i](decoder[i]),batch['mask'])
        if 'inference' in self.output_type:
          probability_from_logit = torch.sigmoid(logit)
          output['probability'] = probability_from_logit

        return output

In [10]:
checks_v1 = ['../input/hubmap-dataset-v2/coat_medium_aug3/coat_nodecoder_best_dice_score_model_aug2_5level_0_stain_medium33.pth','../input/hubmap-dataset-v2/coat_medium_aug3/coat_nodecoder_best_dice_loss_model_aug2_5level_0_stain_medium33.pth','../input/hubmap-dataset-v2/coat_medium_aug3/coat_nodecoder_best_best_dice_score_model_aug2_5level_0_stain_medium33.pth']
checks_v2 = ['../input/hubmap-dataset-v2/coat_medium_aug3/coat_nodecoder_best_dice_score_model_aug2_5level_1_stain_medium33.pth','../input/hubmap-dataset-v2/coat_medium_aug3/coat_nodecoder_best_dice_loss_model_aug2_5level_1_stain_medium33.pth','../input/hubmap-dataset-v2/coat_medium_aug3/coat_nodecoder_best_best_dice_score_model_aug2_5level_1_stain_medium33.pth']
checks_v3 = ['../input/hubmap-dataset-v2/coat_medium_aug3/coat_nodecoder_best_dice_score_model_aug2_5level_2_stain_medium33.pth','../input/hubmap-dataset-v2/coat_medium_aug3/coat_nodecoder_best_dice_loss_model_aug2_5level_2_stain_medium33.pth','../input/hubmap-dataset-v2/coat_medium_aug3/coat_nodecoder_best_best_dice_score_model_aug2_5level_2_stain_medium33.pth']
checks_v4 = ['../input/hubmap-dataset-v2/coat_medium_aug3/coat_nodecoder_best_dice_score_model_aug2_5level_3_stain_medium33.pth','../input/hubmap-dataset-v2/coat_medium_aug3/coat_nodecoder_best_dice_loss_model_aug2_5level_3_stain_medium33.pth','../input/hubmap-dataset-v2/coat_medium_aug3/coat_nodecoder_best_best_dice_score_model_aug2_5level_3_stain_medium33.pth']
checks_v5 = ['../input/hubmap-dataset-v2/coat_medium_aug3/coat_nodecoder_best_dice_score_model_aug2_5level_4_stain_medium33.pth','../input/hubmap-dataset-v2/coat_medium_aug3/coat_nodecoder_best_dice_loss_model_aug2_5level_4_stain_medium33.pth','../input/hubmap-dataset-v2/coat_medium_aug3/coat_nodecoder_best_best_dice_score_model_aug2_5level_4_stain_medium33.pth']
weights = [0.4,0.3,0.3]
net_v1 = load_net(Net_lite_medium,coat_lite_medium(),checks_v1,weights )
net_v2 = load_net(Net_lite_medium,coat_lite_medium(),checks_v2,weights )
net_v3 = load_net(Net_lite_medium,coat_lite_medium(),checks_v3,weights )
net_v4 = load_net(Net_lite_medium,coat_lite_medium(),checks_v4,weights )
net_v5 = load_net(Net_lite_medium,coat_lite_medium(),checks_v5,weights )
all_net = [net_v1,net_v2,net_v3,net_v4,net_v5]

In [11]:
def do_tta_batch(image, organ):
    batch = {
        'image': torch.stack([
            image,
            torch.flip(image,dims=[1]),
            torch.flip(image,dims=[2]),
        ]),
        'organ': torch.Tensor(
            [[organ_to_label[organ]]]*3
        ).long()
    return batch

In [12]:
def undo_tta_batch(probability):
    probability[0] = probability[0]
    probability[1] = torch.flip(probability[1],dims=[1])
    probability[2] = torch.flip(probability[2],dims=[2])
    probability = probability.mean(0, keepdims=True)
    probability = probability[0,0].float()
    return probability

def rle_decode(rle, height, width , fill=255, dtype=np.uint8):
    s = rle.split()
    start, length = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    start -= 1
    mask = np.zeros(height*width, dtype=dtype)
    for i, l in zip(start, length):
        mask[i:i+l] = fill
    mask = mask.reshape(width,height).T
    mask = np.ascontiguousarray(mask)
    return mask

In [14]:
import tifffile
image_size = 768
result = []

def read_tiff(path, scale=None, verbose=0): 
    image = tifffile.imread(path)
    if len(image.shape) == 5:
        image = image.squeeze().transpose(1, 2, 0)
    
    if verbose:
        print(f"[{path}] Image shape: {image.shape}")
    
    if scale:
        new_size = (image.shape[1] // scale, image.shape[0] // scale)
        image = cv2.resize(image, new_size)
        if verbose:
            print(f"[{path}] Resized Image shape: {image.shape}")
        
    mx = np.max(image)
    image = image.astype(np.float32)
    if mx:
        image /= mx # scale image to [0, 1]
    return image



count = 0
for i,d in valid_df.iterrows():
    id = d['id']
    
    if (d['data_source'] in data_source) and (d['organ'] in organ):

        # read the image from the tiff
        tiff_file = tiff_dir +'/%d.tiff'%id
        tiff = read_tiff(tiff_file) 
#         tiff = tiff.astype(np.float32)/255
        H,W,_ = tiff.shape
        
        image = cv2.resize(tiff,dsize=(image_size,image_size),interpolation=cv2.INTER_LINEAR)
        count += 1

        # create tensor batches
        image = image_to_tensor(image, 'rgb')
        batch = { k:v.cuda() for k,v in do_tta_batch(image, d.organ).items() }
#         batch = {}
#         batch['image'] = image.cuda()

        # get the masks probabilities from the pretrained model
        use = 0
        probability = 0
        with torch.no_grad():
            with amp.autocast(enabled = True):

                for n in all_net:
#                         print(use)
#                     for n in net:
                        n.output_type = ['inference']
                        use += 1
                        output = n(batch)
                        probaility = output['probability']
                        # interpolate - resize to the competition image size
                        probability += \
                            F.interpolate(output['probability'], size=(d.img_height,d.img_width),
                                          mode='bilinear',align_corners=False, antialias=True )

                probability = undo_tta_batch(probability / use)

        probability = probability.data.cpu().numpy()
        
        # use the organ threshold to play with the predictions
        p = probability>organ_threshold[d.data_source][d.organ]
        
#         p = probability
        
        # lossless data compression encoding
        rle = rle_encode(p)
        
    else:
        rle = ''

    result.append({ 'id':id, 'rle':rle, })

In [15]:
sub = pd.DataFrame(result)
sub.to_csv('submission.csv',index=False)
sub.head()

,id,rle
0,10078,369 219 2392 219 4415 219 6438 219 8460 220 10...
